# Phase 2: RapidFire AI Classification

This notebook demonstrates the NLP-based job title classification system:

1. **Rule-based classifier** - Fast keyword matching baseline
2. **LLM classifier** - GPT-based classification for nuanced cases
3. **RapidFire framework** - Parallel testing of multiple prompts/models
4. **Evaluation pipeline** - Comprehensive metrics and comparison

In [ ]:
# Setup
import sys
sys.path.insert(0, '..')

import pandas as pd
import numpy as np

# Project imports
from src.models.job_classifier import (
    RuleBasedClassifier, 
    LLMClassifier,
    EnsembleClassifier,
    ClimateCategory
)
from src.models.rapidfire import (
    RapidFireEngine, 
    DataSharder,
    ExperimentConfig,
    ShardStrategy,
    CLASSIFICATION_PROMPTS
)
from src.models.training_data import (
    get_benchmark_dataset,
    get_training_df,
    CLIMATE_SENSITIVE_JOBS,
    CLIMATE_RESILIENT_JOBS
)
from src.models.evaluation import (
    ClassifierEvaluator,
    MetricsCalculator
)

print("Imports successful!")

## 1. Explore Training Data

View the labeled job titles used for training and evaluation.

In [ ]:
# Load training data
df = get_training_df()

print(f"Total labeled titles: {len(df)}")
print(f"\nCategory distribution:")
print(df['category'].value_counts())

print(f"\nIndustry distribution:")
print(df['industry'].value_counts().head(8))

In [ ]:
# Show sample climate-sensitive jobs
print("🌊 Climate-Sensitive Jobs:")
for job in CLIMATE_SENSITIVE_JOBS[:8]:
    print(f"  - {job.title} ({job.industry})")

In [ ]:
# Show sample climate-resilient jobs
print("💻 Climate-Resilient Jobs:")
for job in CLIMATE_RESILIENT_JOBS[:8]:
    print(f"  - {job.title} ({job.industry})")

## 2. Rule-Based Classification

Test the fast keyword-based classifier.

In [ ]:
# Create rule-based classifier
rule_classifier = RuleBasedClassifier(confidence_threshold=0.6)

# Test on sample titles
test_titles = [
    "Surf Instructor",
    "Software Engineer",
    "Commercial Fisherman",
    "Remote Project Manager",
    "Beach Lifeguard",
    "Data Scientist",
    "Restaurant Server",  # Ambiguous
    "Uber Driver"         # Ambiguous
]

print("Rule-Based Classification Results:\n")
print(f"{'Title':<25} {'Category':<20} {'Confidence':<10} {'Keywords'}")
print("-" * 80)

for title in test_titles:
    result = rule_classifier.classify(title)
    keywords = ", ".join(result.matched_keywords[:3])
    print(f"{title:<25} {result.category.value:<20} {result.confidence:.2f}       {keywords}")

## 3. Evaluate Rule-Based Classifier

Test on the benchmark dataset with full metrics.

In [ ]:
# Get benchmark dataset
bench_titles, bench_labels = get_benchmark_dataset()
print(f"Benchmark dataset: {len(bench_titles)} samples")
print(f"  Climate-sensitive: {bench_labels.count('climate_sensitive')}")
print(f"  Climate-resilient: {bench_labels.count('climate_resilient')}")

In [ ]:
# Evaluate
evaluator = ClassifierEvaluator()
result = evaluator.evaluate(
    classifier=rule_classifier,
    test_titles=bench_titles,
    ground_truth=bench_labels,
    model_name="rule_based",
    config_name="threshold_0.6"
)

print("\n📊 Evaluation Metrics:")
print(f"  Accuracy:   {result.metrics.accuracy:.3f}")
print(f"  Precision:  {result.metrics.precision:.3f}")
print(f"  Recall:     {result.metrics.recall:.3f}")
print(f"  F1 Score:   {result.metrics.f1_score:.3f}")
print(f"  Avg Conf:   {result.metrics.avg_confidence:.3f}")

In [ ]:
# Show misclassifications
if result.misclassified_titles:
    print("\n❌ Misclassified Titles:")
    for i, title in enumerate(result.misclassified_titles[:10]):
        idx = result.misclassified_indices[i]
        pred = result.predictions[idx]
        true = result.ground_truth[idx]
        print(f"  '{title}': predicted={pred}, true={true}")

## 4. Compare Different Thresholds

Find the optimal confidence threshold for the rule-based classifier.

In [ ]:
# Create classifiers with different thresholds
thresholds = [0.4, 0.5, 0.6, 0.7, 0.8]

classifiers = {
    f"threshold_{t}": RuleBasedClassifier(confidence_threshold=t)
    for t in thresholds
}

# Compare
comparison = evaluator.compare_classifiers(
    classifiers=classifiers,
    test_titles=bench_titles,
    ground_truth=bench_labels
)

print("\n📈 Threshold Comparison:")
print(comparison[['model', 'accuracy', 'precision', 'recall', 'f1_score', 'uncertain_count']].to_string(index=False))

In [ ]:
# Best threshold
best_idx = comparison['f1_score'].idxmax()
best_threshold = comparison.loc[best_idx, 'model']
best_f1 = comparison.loc[best_idx, 'f1_score']

print(f"\n✅ Best Threshold: {best_threshold} (F1={best_f1:.3f})")

## 5. RapidFire Framework Demo

Use the hyper-parallelization framework to test multiple configurations.

In [ ]:
# Show available prompt templates
print("📝 Available Prompt Templates:\n")
for name, template in CLASSIFICATION_PROMPTS.items():
    preview = template[:100].replace('\n', ' ')
    print(f"  {name}: {preview}...")

In [ ]:
# Initialize RapidFire engine
engine = RapidFireEngine(
    num_shards=2,
    max_workers=2
)

# Add experiments with different thresholds
for threshold in [0.5, 0.6, 0.7]:
    config = ExperimentConfig(
        name=f"rule_thresh_{threshold}",
        prompt_template="N/A",
        model_name="rule_based",
        temperature=threshold,  # Using temperature field for threshold
        metadata={"confidence_threshold": threshold}
    )
    engine.add_experiment(config)

print(f"Added {len(engine.experiments)} experiments")

In [ ]:
# Define classifier factory and classify function
def classifier_factory(config: ExperimentConfig):
    threshold = config.metadata.get("confidence_threshold", 0.6)
    return RuleBasedClassifier(confidence_threshold=threshold)

def classify_func(classifier, title):
    result = classifier.classify(title)
    return result.to_dict()

# Run all experiments
results_df = engine.run_all(
    data=bench_titles,
    classifier_factory=classifier_factory,
    classify_func=classify_func,
    ground_truth=bench_labels
)

print("\n🚀 RapidFire Results:")
print(results_df[['config_name', 'accuracy', 'f1_score', 'total_time']].to_string(index=False))

In [ ]:
# Get best experiment
best = engine.get_best_experiment(metric="f1_score")
if best:
    print(f"\n✅ Best Experiment: {best.config.name}")
    print(f"   Accuracy: {best.accuracy:.2%}")
    print(f"   F1 Score: {best.f1_score:.2%}")
    print(f"   Time: {best.total_time:.2f}s")

## 6. LLM Classification (Optional)

If you have an OpenAI API key, test the LLM-based classifier.

In [ ]:
# Check for OpenAI API key
import os
from dotenv import load_dotenv

load_dotenv()
has_openai_key = bool(os.getenv("OPENAI_API_KEY"))
print(f"OpenAI API Key configured: {has_openai_key}")

if not has_openai_key:
    print("\nTo enable LLM classification:")
    print("1. Get an API key from https://platform.openai.com/")
    print("2. Add OPENAI_API_KEY to your .env file")

In [ ]:
# Test LLM classifier (only if API key is available)
if has_openai_key:
    from src.models.job_classifier import LLMClassifier, LLMClassifierConfig
    
    # Create LLM classifier
    config = LLMClassifierConfig(
        model_name="gpt-4o-mini",
        temperature=0.1
    )
    llm_classifier = LLMClassifier(config)
    
    # Test on a few ambiguous titles
    ambiguous_titles = [
        "Restaurant Server",
        "Hotel Concierge",
        "Uber Driver",
        "Wedding Planner"
    ]
    
    print("\n🤖 LLM Classification Results:")
    for title in ambiguous_titles:
        result = llm_classifier.classify(title)
        print(f"  {title}: {result.category.value} ({result.confidence:.2f})")
        print(f"    Reasoning: {result.reasoning}")
else:
    print("Skip LLM testing (no API key)")

## 7. Summary & Next Steps

### Results
- Rule-based classifier provides fast baseline classification
- RapidFire framework enables parallel testing of configurations
- Key metrics: Accuracy, Precision, Recall, F1 Score

### Next Steps (Phase 3)
1. Integrate classification with workforce data pipeline
2. Build Markov chain models for job transition dynamics
3. Correlate climate events with workforce state changes
4. Develop resilience prediction models

In [ ]:
# Generate final evaluation report
print(evaluator.generate_report())